<a href="https://colab.research.google.com/github/Romainlept/PlayersMap/blob/main/PlayersMap_html.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
HTML("""
<!DOCTYPE html>
<html>

import Globe from 'globe.gl'

</body>
</html>
""")

In [12]:
from IPython.display import HTML

HTML("""
<!DOCTYPE html>
<html>
<head>
  <style> body { margin: 0; } </style>

  <script src="//cdn.jsdelivr.net/npm/globe.gl"></script>
<!--  <script src="../../dist/globe.gl.js"></script>-->
</head>

<body>
  <div id="globeViz"></div>

  <script>
    // Gen random data
    const N = 300;
    const gData = [...Array(N).keys()].map(() => ({
      lat: (Math.random() - 0.5) * 180,
      lng: (Math.random() - 0.5) * 360,
      size: Math.random() / 3,
      color: ['red', 'white', 'blue', 'green'][Math.round(Math.random() * 3)]
    }));

    new Globe(document.getElementById('globeViz'))
      .globeImageUrl('//cdn.jsdelivr.net/npm/three-globe/example/img/earth-night.jpg')
      .pointsData(gData)
      .pointAltitude('size')
      .pointColor('color');

  </script>
</body>
</html>
""")




In [15]:
HTML("""
<!DOCTYPE html>
<html>

<head>
  <style> body { margin: 0; } </style>

  <script src="//cdn.jsdelivr.net/npm/globe.gl"></script>
  <!--<script src="../../dist/globe.gl.js"></script>-->
</head>

<body>
  <div id="globeViz"></div>

  <script>
    // Gen random data
    const N = 20;
    const arcsData = [...Array(N).keys()].map(() => ({
      startLat: (Math.random() - 0.5) * 180,
      startLng: (Math.random() - 0.5) * 360,
      endLat: (Math.random() - 0.5) * 180,
      endLng: (Math.random() - 0.5) * 360,
      color: [['red', 'white', 'blue', 'green'][Math.round(Math.random() * 3)], ['red', 'white', 'blue', 'green'][Math.round(Math.random() * 3)]]
    }));

    new Globe(document.getElementById('globeViz'))
      .globeImageUrl('//cdn.jsdelivr.net/npm/three-globe/example/img/earth-night.jpg')
      .arcsData(arcsData)
      .arcColor('color')
      .arcDashLength(() => Math.random())
      .arcDashGap(() => Math.random())
      .arcDashAnimateTime(() => Math.random() * 4000 + 500);

  </script>
</body>

</body>
</html>
""")

In [16]:
html_code = """
<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8">
  <title>Globe.gl Example</title>
  <style>
    body { margin: 0; overflow: hidden; }
    canvas { display: block; }
    #globeViz { width: 100vw; height: 100vh; }
  </style>
  <script src="https://cdn.jsdelivr.net/npm/globe.gl"></script>
</head>
<body>
  <div id="globeViz"></div>
  <script>
    const N = 20;
    const arcsData = [...Array(N).keys()].map(() => ({
      startLat: (Math.random() - 0.5) * 180,
      startLng: (Math.random() - 0.5) * 360,
      endLat: (Math.random() - 0.5) * 180,
      endLng: (Math.random() - 0.5) * 360,
      color: [['red', 'white', 'blue', 'green'][Math.round(Math.random() * 3)],
              ['red', 'white', 'blue', 'green'][Math.round(Math.random() * 3)]]
    }));

    new Globe(document.getElementById('globeViz'))
      .globeImageUrl('https://cdn.jsdelivr.net/npm/three-globe/example/img/earth-night.jpg')
      .arcsData(arcsData)
      .arcColor('color')
      .arcDashLength(() => Math.random())
      .arcDashGap(() => Math.random())
      .arcDashAnimateTime(() => Math.random() * 4000 + 500);
  </script>
</body>
</html>
"""

# Save the HTML to a file
with open("globe_example.html", "w") as f:
    f.write(html_code)

# Download it so you can open it in your browser
from google.colab import files
files.download("globe_example.html")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
html_code = """
<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8">
  <title>Globe.gl Labeled Arcs</title>
  <style>
    body { margin: 0; overflow: hidden; }
    canvas { display: block; }
    #globeViz { width: 100vw; height: 100vh; }
  </style>
  <script src="https://cdn.jsdelivr.net/npm/globe.gl"></script>
</head>
<body>
  <div id="globeViz"></div>
  <script>
    const nodes = [
      { name: "Pre-Fab Manufacturer - Builder - PP", lat: 35.6762, lng: 139.6503 },
      { name: " - GP/LP - PP", lat: 34.0522, lng: -118.2437 },
      { name: "Toronto - Partner", lat: 43.6532, lng: -79.3832 }
    ];

    const arcsData = [
      {
        startLat: 35.6762,
        startLng: 139.6503,
        endLat: 34.0522,
        endLng: -118.2437,
        color: ['blue', 'green'],
        label: 'Tokyo ➝ LA'
      },
      {
        startLat: 34.0522,
        startLng: -118.2437,
        endLat: 43.6532,
        endLng: -79.3832,
        color: ['green', 'red'],
        label: 'LA ➝ Toronto'
      }
    ];

    const world = new Globe(document.getElementById('globeViz'))
      .globeImageUrl('https://cdn.jsdelivr.net/npm/three-globe/example/img/earth-night.jpg')
      .arcsData(arcsData)
      .arcColor('color')
      .arcLabel('label')
      .arcStroke(0.5)
      .arcDashLength(0.4)
      .arcDashGap(2)
      .arcDashAnimateTime(1500)

      // Add labels to origin points
      .labelsData(nodes)
      .labelLat(d => d.lat)
      .labelLng(d => d.lng)
      .labelText(d => d.name)
      .labelColor(() => 'white')
      .labelSize(0.8)
      .labelDotRadius(0.4)
      .labelResolution(2);
  </script>
</body>
</html>
"""

with open("globe_labeled.html", "w") as f:
    f.write(html_code)

from google.colab import files
files.download("globe_labeled.html")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
HTML("""
<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8">
  <title>Globe.gl - Clickable Nodes</title>
  <style>
    body { margin: 0; overflow: hidden; font-family: Arial; }
    #globeViz { position: absolute; top: 0; left: 0; width: 100vw; height: 100vh; }
    #infoPanel {
      position: absolute;
      right: 20px;
      top: 20px;
      background: rgba(0,0,0,0.8);
      color: white;
      padding: 15px;
      border-radius: 10px;
      width: 250px;
      display: none;
    }
    #infoPanel h3 { margin-top: 0; }
    #infoPanel ul { padding-left: 20px; }
  </style>
  <script src="https://cdn.jsdelivr.net/npm/globe.gl"></script>
</head>
<body>
  <div id="globeViz"></div>
  <div id="infoPanel">
    <h3 id="playerName">Player</h3>
    <p><b>Address:</b> <span id="address"></span></p>
    <p><b>Activity:</b> <span id="activity"></span></p>
    <p><b>Status:</b> <span id="status"></span></p>
    <p><b>Works With:</b></p>
    <ul id="worksWithList"></ul>
  </div>

  <script>
    const players = [
      {
        name: "Tokyo",
        lat: 35.6762,
        lng: 139.6503,
        address: "1-1 Chiyoda, Tokyo, Japan",
        activity: "Real Estate Development",
        status: "Developer",
        worksWith: ["Los Angeles"]
      },
      {
        name: "Los Angeles",
        lat: 34.0522,
        lng: -118.2437,
        address: "456 Sunset Blvd, LA, USA",
        activity: "Venture Investment",
        status: "Investor",
        worksWith: ["Tokyo", "Toronto"]
      },
      {
        name: "Toronto",
        lat: 43.6532,
        lng: -79.3832,
        address: "789 King Street, Toronto, Canada",
        activity: "Joint Venture Projects",
        status: "Partner",
        worksWith: ["Los Angeles"]
      }
    ];

    const arcsData = [
      {
        startLat: 35.6762, startLng: 139.6503,
        endLat: 34.0522, endLng: -118.2437,
        color: ['blue', 'green'],
        label: 'Tokyo ➝ LA'
      },
      {
        startLat: 34.0522, startLng: -118.2437,
        endLat: 43.6532, endLng: -79.3832,
        color: ['green', 'red'],
        label: 'LA ➝ Toronto'
      }
    ];

    const globe = new Globe()(document.getElementById('globeViz'))
      .globeImageUrl('https://cdn.jsdelivr.net/npm/three-globe/example/img/earth-night.jpg')

      // Arc flow
      .arcsData(arcsData)
      .arcColor('color')
      .arcLabel('label')
      .arcDashLength(() => 0.4)
      .arcDashGap(() => 2)
      .arcDashAnimateTime(() => 2000)

      // Labels and Dots
      .labelsData(players)
      .labelLat(d => d.lat)
      .labelLng(d => d.lng)
      .labelText(d => d.name)
      .labelSize(1.3)
      .labelDotRadius(0.5)
      .labelColor(() => 'white')
      .labelLabel(d => `${d.name} (${d.status})`)
      .onLabelClick(showPlayerInfo); // Click handler

    function showPlayerInfo(player) {
      document.getElementById('playerName').innerText = player.name;
      document.getElementById('address').innerText = player.address;
      document.getElementById('activity').innerText = player.activity;
      document.getElementById('status').innerText = player.status;

      const worksWithList = document.getElementById('worksWithList');
      worksWithList.innerHTML = '';
      player.worksWith.forEach(name => {
        const li = document.createElement('li');
        li.textContent = name;
        worksWithList.appendChild(li);
      });

      document.getElementById('infoPanel').style.display = 'block';
    }
  </script>
</body>
</html>
""")

with open("globe_with_player_info.html", "w") as f:
    f.write(html_code)

from google.colab import files
files.download("globe_with_player_info.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [35]:
html_code = """
<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8">
  <title>Globe.gl - Clickable Nodes</title>
  <style>
    body { margin: 0; overflow: hidden; font-family: Arial; }
    #globeViz { position: absolute; top: 0; left: 0; width: 100vw; height: 100vh; }
    #infoPanel {
      position: absolute;
      right: 20px;
      top: 20px;
      background: rgba(0,0,0,0.8);
      color: white;
      padding: 15px;
      border-radius: 10px;
      width: 250px;
      display: none;
    }
    #infoPanel h3 { margin-top: 0; }
    #infoPanel ul { padding-left: 20px; }
  </style>
  <script src="https://cdn.jsdelivr.net/npm/globe.gl"></script>
</head>
<body>
  <div id="globeViz"></div>
  <div id="infoPanel">
    <h3 id="playerName">Player</h3>
    <p><b>Address:</b> <span id="address"></span></p>
    <p><b>Activity:</b> <span id="activity"></span></p>
    <p><b>Status:</b> <span id="status"></span></p>
    <p><b>Works With:</b></p>
    <ul id="worksWithList"></ul>
  </div>

<script>
  const activityColorMap = {
    "Investor": "green",
    "Developer": "red",
    "Educator": "blue",
    "Partner": "purple",
    "Consultant": "orange",
    "Advisor": "yellow"
  };

  const players = [];
  const arcsData = [];

  fetch('https://opensheet.elk.sh/1Z317vxNvOkVkg9f55QfXy2NIY7Eh45yeIzUvDEv/Sheet1')  // <-- the google sheet URL publicly published
    .then(res => res.json())
    .then(data => {
      data.forEach(row => {
        const worksWithArray = row.worksWith ? row.worksWith.split(',').map(name => name.trim()) : [];

        const player = {
          name: row.name,
          lat: parseFloat(row.lat),
          lng: parseFloat(row.lng),
          address: row.address,
          activity: row.activity,
          status: row.status,
          worksWith: worksWithArray
        };

        players.push(player);
      });

      // Now build arcs
      players.forEach(sourcePlayer => {
        sourcePlayer.worksWith.forEach(targetName => {
          const targetPlayer = players.find(p => p.name === targetName);
          if (targetPlayer) {
            arcsData.push({
              startLat: sourcePlayer.lat,
              startLng: sourcePlayer.lng,
              endLat: targetPlayer.lat,
              endLng: targetPlayer.lng,
              color: [
                activityColorMap[sourcePlayer.activity] || 'white',
                activityColorMap[targetPlayer.activity] || 'white'
              ],
              label: `${sourcePlayer.name} ➝ ${targetPlayer.name}`
            });
          }
        });
      });

      // Render the globe
      const globe = new Globe()(document.getElementById('globeViz'))
        .globeImageUrl('https://cdn.jsdelivr.net/npm/three-globe/example/img/earth-night.jpg')
        .arcsData(arcsData)
        .arcColor('color')
        .arcLabel('label')
        .arcDashLength(() => 0.4)
        .arcDashGap(() => 2)
        .arcDashAnimateTime(() => 2000)
        .labelsData(players)
        .labelLat(d => d.lat)
        .labelLng(d => d.lng)
        .labelText(d => d.name)
        .labelSize(1.3)
        .labelDotRadius(0.5)
        .labelColor(() => 'white')
        .labelLabel(d => `${d.name} (${d.status})`)
        .onLabelClick(showPlayerInfo);
    });

  function showPlayerInfo(player) {
    document.getElementById('playerName').innerText = player.name;
    document.getElementById('address').innerText = player.address;
    document.getElementById('activity').innerText = player.activity;
    document.getElementById('status').innerText = player.status;

    const worksWithList = document.getElementById('worksWithList');
    worksWithList.innerHTML = '';
    player.worksWith.forEach(name => {
      const li = document.createElement('li');
      li.textContent = name;
      worksWithList.appendChild(li);
    });

    document.getElementById('infoPanel').style.display = 'block';
  }
</script>
</body>
</html>
"""

with open("interactive_globe.html", "w") as f:
    f.write(html_code)

from google.colab import files
files.download("interactive_globe.html")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [36]:
python -m http.server 8000


SyntaxError: invalid syntax (<ipython-input-36-67576644b2a6>, line 1)